In [1]:
!pip install -Uqq scikit-learn nltk

In [1]:
import json
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import random

#### Loading Data

In [127]:
domain_name = 'ABSA_Dataset_train'
with open(domain_name+'.jsonl', 'r') as file:
    lines = file.readlines()

raw_dataset = []

for i,line in enumerate(lines):
    try:
        data = json.loads(line.replace('\'', '\"'))
        raw_dataset.append(data)
        data['review'] = data['review'].replace('\"','')
        data['review'] = data['review'].replace("\'",'')
    except:
        print(i)

In [128]:
raw_dataset_size = len(raw_dataset)
print(f'dataset size is {raw_dataset_size}')

dataset size is 13617


#### Setting review ids/ extracting aspects

In [98]:
aspects_set = set()

In [99]:
for i, example in enumerate(raw_dataset):
    example['review_id'] = i+1
    aspects = list(example['aspects'].keys())
    aspects_set.update(aspects)

In [100]:
aspects_set

{'بازی', 'داستان', 'صحنه', 'صدا', 'فیلمبرداری', 'موسیقی', 'کارگردانی'}

In [101]:
raw_dataset[51]

{'review': 'فیلمی پر کشش و دوست داشتنی   اما تلخ و آموزنده...   بازی آقای معادی اصلا در اندازه فیلم نبود و خانم جاوهریان هم گاهی به انچه باید می بود، نمی رسید.   بهترین بازیگر این فیلم، همان کودک جانسپرده بود که کاملا همراه بودو به خوبی خوابید...',
 'sentiment': '1',
 'category': ' ملبورن',
 'aspects': {'بازی': '3'},
 'review_id': 52}

In [102]:
for entry in raw_dataset:
    if 'category' not in entry:
        print(entry['review_id'])
        raise Exception(entry['review_id'] + " does not have any category associated")

#### Defining Questions for aspects

In [103]:
aspects_candidate_words = {
    # For Foods"
    "ارزش خرید" : "قیمت و ارزش خرید",
    "بسته بندی" : "بسته بندی و نگهداری",
    "ارسال" : "ارسال و حمل و نقل",
    "کیفیت" : "کیفیت و تازگی",
    "ارزش غذایی" : "سلامت و ارزش غذایی",
    "طعم" : "عطر، بو، و طعم",
    # For Movies:
    "صدا" : "صداگذاری و جلوه های صوتی",
    "موسیقی" : "موسیقی",
    "داستان" : "داستان، فیلمنامه، دیالوگ ها و موضوع",
    "صحنه" : "گریم، طراحی صحنه و جلوه های ویژه ی بصری",
    "کارگردانی" : "تهیه، تدوین، کارگردانی و ساخت",
    "فیلمبرداری" : "فیلمبرداری و تصویربرداری",
    "بازی" : "شخصیت پردازی، بازیگردانی و بازی بازیگران"
}

In [104]:
aspects_questions = {}
for aspect in list(aspects_set):
    aspects_questions[aspect] = f'نظر شما در مورد {aspects_candidate_words[aspect]} این محصول چیست؟'

In [105]:
general_aspect_label = 'کلی'
aspects_questions[general_aspect_label] = 'نظر شما به صورت کلی در مورد این محصول چیست؟'

In [106]:
aspects_questions

{'فیلمبرداری': 'نظر شما در مورد فیلمبرداری و تصویربرداری این محصول چیست؟',
 'صدا': 'نظر شما در مورد صداگذاری و جلوه های صوتی این محصول چیست؟',
 'کارگردانی': 'نظر شما در مورد تهیه، تدوین، کارگردانی و ساخت این محصول چیست؟',
 'بازی': 'نظر شما در مورد شخصیت پردازی، بازیگردانی و بازی بازیگران این محصول چیست؟',
 'موسیقی': 'نظر شما در مورد موسیقی این محصول چیست؟',
 'داستان': 'نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع این محصول چیست؟',
 'صحنه': 'نظر شما در مورد گریم، طراحی صحنه و جلوه های ویژه ی بصری این محصول چیست؟',
 'کلی': 'نظر شما به صورت کلی در مورد این محصول چیست؟'}

### Spliting Data

In [107]:
NONE_LABEL = -3

In [108]:
X = []
y = []
for rec in raw_dataset:
    X.append ({k:rec[k] for k in ('review','review_id','aspects','category') if k in rec})
    y.append(rec['sentiment'])

In [109]:
X[0],y[0]

({'review': 'یکی از دوستان اشاره خوبی داشتن   چقد موسیقی حماسی و بی مورد؟  فقط میتونن بگم این سوژه اگه به گروه و\u200c کَست بهتری داده میشه نتیجه کار خیلی قابل قبول تر از این میشد',
  'review_id': 1,
  'aspects': {'موسیقی': '-1', 'بازی': '-1'},
  'category': ' ماهورا'},
 '-1')

In [110]:
test_dev_size_dic = {'movie':(0.2,0.11),'food':(0.1,0.11)}

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_dev_size_dic[domain_name][0], random_state=12, stratify=y)

In [112]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_dev_size_dic[domain_name][1], random_state=12, stratify=y_train)

In [113]:
len(X_train),len(X_test),len(X_valid)

(359, 102, 45)

In [114]:
X_train[0],y_train[0]

({'review': 'بدترین بازی ها از بهترین بازیگرا در یکی از بدترین فیلم های جشنواره!',
  'review_id': 56,
  'aspects': {'بازی': '-2'},
  'category': ' مردی بدون سایه'},
 '-2')

### calculating similarities:

In [115]:
# bleu_avg = 0
# for entry_1 in X_train:
#     for entry_2 in X_test:
#         bleu_avg += nltk.translate.bleu_score.sentence_bleu(references=[entry_1['review']],hypothesis=entry_2['review'])
        
# bleu_avg = bleu_avg / (len(X_train)*len(X_test))
# print(f"BLEU = {bleu_avg}")

#### Putting X and y together:

In [116]:
raw_dataset_dic = dict()
for entry in zip(X_train,y_train):
    entry[0].update({'sentiment':entry[1]})
    
for entry in zip(X_test,y_test):
    entry[0].update({'sentiment':entry[1]})
    
for entry in zip(X_valid,y_valid):
    entry[0].update({'sentiment':entry[1]})

In [117]:
raw_dataset_dic = {
    'train': X_train,
    'test': X_test,
    'dev': X_valid
}

In [118]:
assert sum([len(dataset) for dataset in raw_dataset_dic.values()]) == raw_dataset_size

#### Creating new QA-ABSA dataset

In [119]:
def gen_question(aspect,aspects_questions,example,domain_name:str):
    
    if domain_name.lower()=='food':
        question = aspects_questions[aspect].replace('محصول',example['category'])
        
    elif domain_name.lower()=='movie':
        question = aspects_questions[aspect].replace('این محصول','فیلم ' + example['category'])

    else:
        raise Exception('Domain is not supported')
        
    return question

In [120]:
dataset_ABSA = {'train':list(),'dev':list(),'test':list()}
for dataset_name, dataset in raw_dataset_dic.items():

    for example in dataset:
        
        product_aspects = example['aspects'].keys()

        #aspect sentiments
        i=1
        for aspect in aspects_set:
            entry = {'review': example['review'],
                     'review_id': str(example['review_id']),
                     'example_id': str(i),
                     'question': gen_question(aspect,aspects_questions,example,domain_name),
                     'category': example['category'],
                     'aspect': aspect,
                     'label': str(example['aspects'][aspect] if aspect in product_aspects else NONE_LABEL)
                    }
            
            dataset_ABSA[dataset_name].append(entry)
            i+=1

        # overal sentiment
        entry = {'review': example['review'],
                 'review_id': str(example['review_id']),
                 'example_id': str(i),
                 'question': gen_question(general_aspect_label,aspects_questions,example,domain_name),
                 'category': example['category'],
                 'aspect': general_aspect_label,
                 'label': str(example['sentiment'])
                }
        
        dataset_ABSA[dataset_name].append(entry)

In [121]:
dataset_ABSA['train'][0:10]

[{'review': 'بدترین بازی ها از بهترین بازیگرا در یکی از بدترین فیلم های جشنواره!',
  'review_id': '56',
  'example_id': '1',
  'question': 'نظر شما در مورد فیلمبرداری و تصویربرداری فیلم  مردی بدون سایه چیست؟',
  'category': ' مردی بدون سایه',
  'aspect': 'فیلمبرداری',
  'label': '-3'},
 {'review': 'بدترین بازی ها از بهترین بازیگرا در یکی از بدترین فیلم های جشنواره!',
  'review_id': '56',
  'example_id': '2',
  'question': 'نظر شما در مورد صداگذاری و جلوه های صوتی فیلم  مردی بدون سایه چیست؟',
  'category': ' مردی بدون سایه',
  'aspect': 'صدا',
  'label': '-3'},
 {'review': 'بدترین بازی ها از بهترین بازیگرا در یکی از بدترین فیلم های جشنواره!',
  'review_id': '56',
  'example_id': '3',
  'question': 'نظر شما در مورد تهیه، تدوین، کارگردانی و ساخت فیلم  مردی بدون سایه چیست؟',
  'category': ' مردی بدون سایه',
  'aspect': 'کارگردانی',
  'label': '-3'},
 {'review': 'بدترین بازی ها از بهترین بازیگرا در یکی از بدترین فیلم های جشنواره!',
  'review_id': '56',
  'example_id': '4',
  'question': 'نظ

#### Adding ID labeles

In [122]:
for dataset_name, dataset in dataset_ABSA.items():
    for i, example in enumerate(dataset):
#         example['example_id'] = str(int(i+1))
        example['guid'] = f'{domain_name}-{dataset_name}-r{example["review_id"]}-e{example["example_id"]}'

#### Saving Data

In [123]:
for dataset_name, dataset in dataset_ABSA.items():
    
    with open(f'{domain_name}_{dataset_name}.jsonl', 'w') as f:
        
        for example in dataset:
            
#             f.write("%s\n" % annotation)
            f.write(f'{example}\n')
print('DONE')

DONE


### Merging the two datasets:

In [125]:
for dataset_split_name in ['train','test','dev']:
    with open (f'ABSA_Dataset_{dataset_split_name}.jsonl','w') as out_file:
        for domain in ['food','movie']:
            with open (f'{domain}_{dataset_split_name}.jsonl') as in_file:
                for line in in_file.readlines():
                    out_file.write(line.strip()+'\n')